In [3]:
import os
import numpy as np
import json
import torch
import cv2
import torchvision
import shutil
import albumentations as A
import torchvision.transforms as transforms
import random
import csv
import pandas as pd
import warnings

from datasets import load_dataset, Dataset
from glob import glob
from tqdm import tqdm
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader, Dataset
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from PIL import Image, ImageDraw
from pathlib import Path
from ultralytics import YOLO

warnings.filterwarnings("ignore")

## Работа с RCNN

In [447]:
augmentations = A.Compose([
    A.SomeOf([
        A.RandomBrightnessContrast(),
        A.GaussianBlur(blur_limit=(3, 5)),
        A.GaussNoise(var_limit=(10, 50)),
        A.Perspective(scale=(0.05, 0.1))
    ], n=2), 
    A.Resize(512, 512),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

In [449]:
def augment_image_with_boxes(image, boxes, labels):
    augmented = augmentations(image=np.array(image), bboxes=boxes, labels=labels.tolist())
    return augmented['image'], augmented['bboxes'], augmented['labels']

In [451]:
class TextDataset(Dataset):
    def __init__(self, image_dir, annotations_file, transforms=None):
        self.image_dir = image_dir
        self.transforms = transforms

        with open(annotations_file) as f:
            coco_data = json.load(f)

        self.images = {img['id']: img for img in coco_data['images']}
        self.annotations = coco_data['annotations']

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_info = list(self.images.values())[idx]
        img_path = os.path.join(self.image_dir, image_info['file_name'])
        image = Image.open(img_path).convert("RGB")

        image_id = image_info['id']
        boxes = []
        labels = []

        for annotation in self.annotations:
            if annotation['image_id'] == image_id:
                x_min, y_min, width, height = annotation['bbox']
                boxes.append([x_min, y_min, x_min + width, y_min + height])
                labels.append(annotation['category_id'])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        if self.transforms:
            image, boxes, labels = self.transforms(image, boxes, labels)
        
        target = {
            'boxes': boxes,
            'labels': torch.tensor(labels, dtype=torch.int64)
        }
        return image, target

In [453]:
batch_size = 1
dataset = TextDataset('D:/datasets/hack_t1_ds', 'D:/datasets/hack_t1_ds/result.json', transforms=augment_image_with_boxes)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [455]:
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model

In [457]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = get_model(num_classes=2)  # 2 класса: печатный и рукописный текст
model.to(device)

# Оптимизатор
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=0.005)

# Функция обучения
num_epochs = 350

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, targets in data_loader:
        images = [img.to(device) / 255.0 for img in images]
        targets_f = [
            {
                'boxes': targets['boxes'][i].to(device),
                'labels': targets['labels'][i].to(device)
            } for i in range(batch_size)
        ]
        loss_dict = model(images, targets_f)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        total_loss += losses.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(data_loader)}")

D:\anaconda\Lib\site-packages\torch\nn\modules\linear.py:114: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\Context.cpp:156.)
  return F.linear(input, self.weight, self.bias)
D:\anaconda\Lib\site-packages\torch\autograd\__init__.py:251: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic becau

Epoch 1, Loss: 54069006072202.59
Epoch 2, Loss: 3159938593.3636365
Epoch 3, Loss: 8567452970.984848
Epoch 4, Loss: 41376492423.63636
Epoch 5, Loss: 65931577402.181816
Epoch 6, Loss: 3823931574079.0303
Epoch 7, Loss: 218840433252.84848
Epoch 8, Loss: 1731221996384.2424
Epoch 9, Loss: 22299186775877.816
Epoch 10, Loss: 15571682848457.697
Epoch 11, Loss: 50479532079104.0
Epoch 12, Loss: nan
Epoch 13, Loss: nan
Epoch 14, Loss: 126704490603861.33
Epoch 15, Loss: 1785873352709461.2
Epoch 16, Loss: 468536018775815.75
Epoch 17, Loss: 2334422785315685.0
Epoch 18, Loss: 2295787397836924.0
Epoch 19, Loss: 2.3363492198039626e+19
Epoch 20, Loss: 2.8448679238996637e+17
Epoch 21, Loss: 1.956578717713357e+17
Epoch 22, Loss: 9454696256948658.0
Epoch 23, Loss: 4420190939238772.5
Epoch 24, Loss: 1767555603040194.0
Epoch 25, Loss: 5678192539213.576
Epoch 26, Loss: 108855320953452.61
Epoch 27, Loss: 25209606571969.938
Epoch 28, Loss: 2791437767524.8486
Epoch 29, Loss: 30905930480.484848
Epoch 30, Loss: 517

In [459]:
def predict_boxes_and_labels(image_path, model, device):
    image = Image.open(image_path).convert("RGB")
    eval_augmentations = A.Compose([
        A.Resize(512, 512),
        ToTensorV2()
    ])
    model.eval()
    with torch.no_grad():
        preprocessing_image = eval_augmentations(image=np.array(image))
        normalized_image = preprocessing_image['image'] / 255.0
        normalized_image = normalized_image.unsqueeze(0)
        normalized_image = normalized_image.to(device)  
        predictions = model(normalized_image)

    print(predictions)
    boxes = predictions[0]['boxes']
    labels = predictions[0]['labels']
    scores = predictions[0]['scores']

    threshold = 0.3
    filtered_boxes = boxes[scores > threshold].cpu().numpy()
    filtered_labels = labels[scores > threshold].cpu().numpy()

    return filtered_boxes, filtered_labels

In [461]:
image_path = 'D:/datasets/hack_t1_ds/images/9390a1ce-3_t_2.jpg'
boxes, labels = predict_boxes_and_labels(image_path, model, device)

[{'boxes': tensor([], device='cuda:0', size=(0, 4)), 'labels': tensor([], device='cuda:0', dtype=torch.int64), 'scores': tensor([], device='cuda:0')}]


In [431]:
image = Image.open(image_path).convert("RGB")
draw = ImageDraw.Draw(image)

for box, label in zip(boxes, labels):
    x_min, y_min, x_max, y_max = box.tolist()
    draw.rectangle([x_min, y_min, x_max, y_max], outline='red', width=2)
    draw.text((x_min, y_min), str(label.item()), fill='red')

image.show() 

## Работа с YOLO

In [465]:
model = YOLO('yolov8n.pt')
model.train(data='D:/datasets/yolo8_ds/data.yaml', epochs=80, augment=True)

Ultralytics 8.3.23  Python-3.11.5 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:/datasets/yolo8_ds/data.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

D:\anaconda\Lib\site-packages\ultralytics\utils\torch_utils.py:262: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\Context.cpp:156.)
  fusedconv.weight.copy_(torch.mm(w_bn, w_conv).view(fusedconv.weight.shape))
D:\anaconda\Lib\site-packages\ultralytics\utils\torch_utils.py:267: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but 

AMP: checks passed 


train: Scanning D:\datasets\yolo8_ds\labels\train.cache... 33 images, 0 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

D:\anaconda\Lib\site-packages\ultralytics\utils\torch_utils.py:262: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\Context.cpp:156.)
  fusedconv.weight.copy_(torch.mm(w_bn, w_conv).view(fusedconv.weight.shape))
D:\anaconda\Lib\site-packages\ultralytics\utils\torch_utils.py:267: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but 

Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.27it/s]


                   all          1         19      0.921      0.953      0.966      0.682
         handwrite_rus          1         16          1      0.906      0.937      0.555
             signature          1          3      0.843          1      0.995      0.809
Speed: 0.0ms preprocess, 425.6ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to runs\detect\train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001DD41EA6C90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [469]:
model.save("yolov8_weights.pt")

In [475]:
predict('D:/datasets/yolo8_ds/images/train/b3a51f3e-1_pass_1.png')


image 1/1 D:\datasets\yolo8_ds\images\train\b3a51f3e-1_pass_1.png: 640x480 13 handwrite_russ, 2 signatures, 430.9ms
Speed: 34.5ms preprocess, 430.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


### Функции трасформации данных в YOLO формат

In [569]:
def convert_to_yolo_format(image_folder, annotation_folder, output_folder, train_split=0.8):
    img_output_folder = os.path.join(output_folder, 'images')
    lbl_output_folder = os.path.join(output_folder, 'labels')
    train_img_folder = os.path.join(img_output_folder, 'train')
    val_img_folder = os.path.join(img_output_folder, 'val')
    train_lbl_folder = os.path.join(lbl_output_folder, 'train')
    val_lbl_folder = os.path.join(lbl_output_folder, 'val')

    for folder in [img_output_folder, lbl_output_folder, train_img_folder, val_img_folder, train_lbl_folder, val_lbl_folder]:
        os.makedirs(folder, exist_ok=True)

    img_paths = list(Path(image_folder).glob('*'))
    
    random.shuffle(img_paths)
    split_index = int(len(img_paths) * train_split)
    train_imgs = img_paths[:split_index]
    val_imgs = img_paths[split_index:]

    def convert_bbox_to_yolo(size, box):
        x1, y1 = box[0]
        x2, y2 = box[1]
        dw = 1. / size[0]
        dh = 1. / size[1]

        x = (x1 + x2) / 2.0 * dw
        y = (y1 + y2) / 2.0 * dh
        w = (x2 - x1) * dw
        h = (y2 - y1) * dh
        return (x, y, w, h)
    
    def process_images(img_paths, label_folder, img_dest_folder):
        for img_path in img_paths:
            img_name = img_path.stem
            json_path = os.path.join(annotation_folder, f"{img_name}.json")

            if not os.path.exists(json_path):
                print(f"Annotation not found for {img_name}")
                continue

            with open(json_path, 'r', encoding='utf-8') as f:
                annotations = json.load(f)

            with Image.open(img_path) as img:
                img = img.convert("RGB")
                width, height = img.size

                label_data = []
                for item in annotations:
                    class_id = 1 if item['signature'] else 0
                    box = item['coordinates']
                    x, y, w, h = convert_bbox_to_yolo((width, height), box)
                    label_data.append(f"{class_id} {x} {y} {w} {h}")

                label_path = os.path.join(label_folder, f"{img_name}.txt")
                with open(label_path, 'w') as label_file:
                    label_file.write("\n".join(label_data))

                img_dest = os.path.join(img_dest_folder, f"{img_name}.jpg")
                img.save(img_dest)

    process_images(train_imgs, train_lbl_folder, train_img_folder)
    process_images(val_imgs, val_lbl_folder, val_img_folder)

    data_yaml = f"""path: {output_folder}
train: images/train
val: images/val

names:
  0: handwrite_rus
  1: signature
"""

    with open(os.path.join(output_folder, 'data.yaml'), 'w') as yaml_file:
        yaml_file.write(data_yaml)

In [575]:
convert_to_yolo_format("D:/datasets/hack/hack_ds/_public", "D:/datasets/hack/annotations", "D:/datasets/OVER_DS_DETECTION")

In [541]:
augmentations = A.Compose([
    A.SomeOf([
        A.RandomBrightnessContrast(),
        A.GaussianBlur(blur_limit=(3, 5)),
        A.GaussNoise(var_limit=(10, 50)),
    ], n=1)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids', 'contents']))

def create_augmented_data(image_folder, annotation_folder, output_folder, num_augmentations=3, val_split=0.2):
    os.makedirs(output_folder, exist_ok=True)
    image_paths = glob(os.path.join(image_folder, "*.*"))  # Все форматы файлов

    aug_images = []
    aug_annotations = []

    for image_path in tqdm(image_paths):
        image = cv2.imread(image_path)
        if image is None:
            print(f"Файл {image_path} не является изображением и пропускается.")
            continue

        image_name = os.path.basename(image_path)
        base_name = os.path.splitext(image_name)[0]

        annotation_path = os.path.join(annotation_folder, f"{base_name}.json")
        if not os.path.exists(annotation_path):
            print(f"Аннотация для {image_name} не найдена.")
            continue

        with open(annotation_path, 'r', encoding='utf-8') as f:
            annotations = json.load(f)

        bboxes = []
        category_ids = []
        contents = []
        for ann in annotations:
            x_min, y_min = ann["coordinates"][0]
            x_max, y_max = ann["coordinates"][1]
            bboxes.append([x_min, y_min, x_max, y_max])
            contents.append(ann['content'])
            category_ids.append(1 if ann["signature"] else 0)

        for i in range(num_augmentations):
            augmented = augmentations(image=image, bboxes=bboxes, category_ids=category_ids, contents=contents)
            aug_image = augmented['image']
            aug_bboxes = augmented['bboxes']
            aug_category_ids = augmented['category_ids']
            aug_contents = augmented['contents']
            
            aug_image_name = f"{base_name}_aug_{i}.jpg"
            aug_image_path = os.path.join(output_folder, "images", aug_image_name)
            os.makedirs(os.path.dirname(aug_image_path), exist_ok=True)
            cv2.imwrite(aug_image_path, aug_image)
            aug_images.append(aug_image_path)

            aug_annotations_data = []
            for bbox, cat_id, cont in zip(aug_bboxes, aug_category_ids, aug_contents):
                x_min, y_min, x_max, y_max = bbox
                aug_annotations_data.append({
                    "signature": bool(cat_id),
                    "content": cont,
                    "coordinates": [[x_min, y_min], [x_max, y_max]]
                })

            aug_annotation_name = f"{base_name}_aug_{i}.json"
            aug_annotation_path = os.path.join(output_folder, "annotations", aug_annotation_name)
            os.makedirs(os.path.dirname(aug_annotation_path), exist_ok=True)
            with open(aug_annotation_path, 'w', encoding='utf-8') as f:
                json.dump(aug_annotations_data, f, ensure_ascii=False, indent=4)
            aug_annotations.append(aug_annotation_path)

    train_images, val_images, train_annots, val_annots = train_test_split(
        aug_images, aug_annotations, test_size=val_split, random_state=42
    )

    for img_path, annot_path in zip(train_images, train_annots):
        train_image_dir = os.path.join(output_folder, "train", "images")
        train_annot_dir = os.path.join(output_folder, "train", "annotations")
        os.makedirs(train_image_dir, exist_ok=True)
        os.makedirs(train_annot_dir, exist_ok=True)
        shutil.move(img_path, os.path.join(train_image_dir, os.path.basename(img_path)))
        shutil.move(annot_path, os.path.join(train_annot_dir, os.path.basename(annot_path)))

    for img_path, annot_path in zip(val_images, val_annots):
        val_image_dir = os.path.join(output_folder, "val", "images")
        val_annot_dir = os.path.join(output_folder, "val", "annotations")
        os.makedirs(val_image_dir, exist_ok=True)
        os.makedirs(val_annot_dir, exist_ok=True)
        shutil.move(img_path, os.path.join(val_image_dir, os.path.basename(img_path)))
        shutil.move(annot_path, os.path.join(val_annot_dir, os.path.basename(annot_path)))

    print("Аугментация и разбиение завершены.")

In [543]:
# Пример вызова функции
create_augmented_data(
    image_folder="D:/datasets/hack/hack_ds/_public",
    annotation_folder="D:/datasets/hack/annotations",
    output_folder="D:/datasets/OVER_DS_DETECTION",
    num_augmentations=4,
    val_split=0.2
)

100%|██████████| 33/33 [00:13<00:00,  2.49it/s]

Аугментация и разбиение завершены.


In [545]:
def prepare_yolo_annotations(dataset_path, output_path):
    train_img_dir = os.path.join(output_path, 'images', 'train')
    val_img_dir = os.path.join(output_path, 'images', 'val')
    train_lbl_dir = os.path.join(output_path, 'labels', 'train')
    val_lbl_dir = os.path.join(output_path, 'labels', 'val')

    # Создаем папки для train и val
    for dir_path in [train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
        os.makedirs(dir_path, exist_ok=True)

    def convert_bbox_to_yolo(size, box):
        x_min, y_min = box[0]
        x_max, y_max = box[1]
        dw = 1. / size[0]
        dh = 1. / size[1]
        x = (x_min + x_max) / 2.0 * dw
        y = (y_min + y_max) / 2.0 * dh
        w = (x_max - x_min) * dw
        h = (y_max - y_min) * dh
        return x, y, w, h

    def process_data(image_folder, annotation_folder, img_dest_folder, lbl_dest_folder):
        for img_path in Path(image_folder).glob("*"):
            img_name = img_path.stem
            json_path = os.path.join(annotation_folder, f"{img_name}.json")

            if not os.path.exists(json_path):
                print(f"Annotation not found for {img_name}")
                continue

            try:
                with open(json_path, 'r', encoding='utf-8') as f:
                    annotations = json.load(f)
            except UnicodeDecodeError:
                with open(json_path, 'r', encoding='cp1251') as f:
                    annotations = json.load(f)


            try:
                with Image.open(img_path) as img:
                    img = img.convert("RGB")
                    width, height = img.size

                    label_data = []
                    for item in annotations:
                        class_id = 1 if item['signature'] else 0
                        box = item['coordinates']
                        x, y, w, h = convert_bbox_to_yolo((width, height), box)
                        label_data.append(f"{class_id} {x} {y} {w} {h}")

                    label_path = os.path.join(lbl_dest_folder, f"{img_name}.txt")
                    with open(label_path, 'w') as label_file:
                        label_file.write("\n".join(label_data))

                    img_dest = os.path.join(img_dest_folder, f"{img_name}.jpg")
                    img.save(img_dest)

            except Exception as e:
                print(f"Error processing {img_name}: {e}")

    train_images_folder = os.path.join(dataset_path, 'train', 'images')
    train_annotations_folder = os.path.join(dataset_path, 'train', 'annotations')
    val_images_folder = os.path.join(dataset_path, 'val', 'images')
    val_annotations_folder = os.path.join(dataset_path, 'val', 'annotations')

    process_data(train_images_folder, train_annotations_folder, train_img_dir, train_lbl_dir)
    process_data(val_images_folder, val_annotations_folder, val_img_dir, val_lbl_dir)

    data_yaml_content = f"""
path: {output_path}
train: images/train
val: images/val

names:
  0: handwrite_rus
  1: signature
"""
    with open(os.path.join(output_path, 'data.yaml'), 'w') as yaml_file:
        yaml_file.write(data_yaml_content.strip())

    print("YOLO dataset preparation completed with annotations.")

In [547]:
# Пример вызова функции
prepare_yolo_annotations("D:/datasets/hack/OVER_DS_DETECTION", "D:/datasets/hack/OVER_YOLO8_DETECTION")

YOLO dataset preparation completed with annotations.


In [581]:
model = YOLO('yolov8n.pt')
model.train(data='D:/datasets/hack/OVER_DS_DETECTION/data.yaml', epochs=80)

Ultralytics 8.3.23  Python-3.11.5 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:/datasets/hack/OVER_DS_DETECTION/data.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

train: Scanning D:\datasets\hack\OVER_DS_DETECTION\labels\train... 26 images, 0 backgrounds, 0 corrupt: 100%|██████████| 26/26 [00:00<00:00, 534.31it/s]

train: New cache created: D:\datasets\hack\OVER_DS_DETECTION\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning D:\datasets\hack\OVER_DS_DETECTION\labels\val... 7 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7/7 [00:00<00:00, 538.47it/s]

val: New cache created: D:\datasets\hack\OVER_DS_DETECTION\labels\val.cache


...

                   all          7        102      0.663      0.766       0.77       0.46



80 epochs completed in 0.037 hours.
Optimizer stripped from runs\detect\train13\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train13\weights\best.pt, 6.3MB

Validating runs\detect\train13\weights\best.pt...
Ultralytics 8.3.23  Python-3.11.5 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 12.07it/s]


                   all          7        102      0.828      0.641      0.776      0.455
         handwrite_rus          7         88      0.747      0.571      0.666      0.401
             signature          7         14      0.909      0.711      0.886       0.51
Speed: 0.1ms preprocess, 1.9ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs\detect\train13


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001DC419D0E50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [13]:
def predict(image_path):
    results = model.predict(source=image_path, conf=0.4)
    for r in results:
        r.show()

res = predict('D:/datasets/hack/hack_ds/_public/4_t_1.png')


image 1/1 D:\datasets\hack\hack_ds\_public\4_t_1.png: 480x640 26 handwrite_russ, 180.0ms
Speed: 4.0ms preprocess, 180.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


In [591]:
model.save('yolo8n_weights_v2.pt')

In [7]:
model = YOLO('yolo8n_weights_v2.pt')
results = model.val(data="D:/datasets/hack/OVER_DS_DETECTION/data.yaml", save_json=True)
print("mAP@0.5:", results.box.map50)

Ultralytics 8.3.23  Python-3.11.5 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)


val: Scanning D:\datasets\hack\OVER_DS_DETECTION\labels\val.cache... 7 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7/7 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.20s/it]


                   all          7        102      0.828      0.641      0.773       0.46
         handwrite_rus          7         88      0.747      0.571      0.666      0.402
             signature          7         14      0.909       0.71       0.88      0.517
Speed: 1.0ms preprocess, 32.9ms inference, 0.0ms loss, 25.0ms postprocess per image
Saving runs\detect\val\predictions.json...
Results saved to runs\detect\val
mAP@0.5: 0.7727488010976515


## Этап распознования текста

In [599]:
def extract_and_save_images(annotations_folder, images_folder, output_folder, tsv_path):
    os.makedirs(output_folder, exist_ok=True)

    with open(tsv_path, mode='a', newline='', encoding='utf-8') as tsv_file:
        tsv_writer = csv.writer(tsv_file, delimiter='\t')
        tsv_writer.writerow(["file_name", "transcribe"])

        for annotation_file in os.listdir(annotations_folder):
            if annotation_file.endswith(".json"):
                annotation_path = os.path.join(annotations_folder, annotation_file)
                base_name = os.path.splitext(annotation_file)[0]
                
                image_path = None
                for ext in [".jpg", ".jpeg", ".png"]:
                    potential_path = os.path.join(images_folder, f"{base_name}{ext}")
                    if os.path.exists(potential_path):
                        image_path = potential_path
                        break

                if not image_path:
                    print(f"Image for {base_name} not found, skipping.")
                    continue

                with open(annotation_path, 'r', encoding='utf-8') as f:
                    annotations = json.load(f)
                image = Image.open(image_path)

                for i, item in enumerate(annotations):
                    if not item.get("signature", False):  # signature == False
                        coordinates = item["coordinates"]
                        content = item["content"]

                        x_min, y_min = coordinates[0]
                        x_max, y_max = coordinates[1]
                        cropped_image = image.crop((x_min, y_min, x_max, y_max))

                        output_image_name = f"{base_name}_{i}.png"
                        output_image_path = os.path.join(output_folder, output_image_name)
                        cropped_image.save(output_image_path, format="PNG")

                        tsv_writer.writerow([output_image_name, content])

                print(f"Processed {base_name} with {len(annotations)} items.")

    print(f"Processing completed. Images saved to {output_folder} and details logged in {tsv_path}.")

In [605]:
extract_and_save_images(
    annotations_folder="D:/datasets/hack/annotations",
    images_folder="D:/datasets/hack/hack_ds/_public",
    output_folder="D:/datasets/hack/handwritten_OCR/train",
    tsv_path="D:/datasets/hack/handwritten_OCR/train.tsv"
)

Processed 0_kop_0 with 23 items.
Processed 1_pass_1 with 24 items.
Processed 1_pass_2 with 19 items.
Processed 1_pass_3 with 20 items.
Processed 1_pass_4 with 14 items.
Processed 1_pass_5 with 15 items.
Processed 2_p_1 with 23 items.
Processed 2_p_10 with 8 items.
Processed 2_p_11 with 11 items.
Processed 2_p_12 with 7 items.
Processed 2_p_13 with 23 items.
Processed 2_p_14 with 8 items.
Processed 2_p_15 with 24 items.
Processed 2_p_16 with 9 items.
Processed 2_p_17 with 7 items.
Processed 2_p_2 with 8 items.
Processed 2_p_3 with 9 items.
Processed 2_p_4 with 7 items.
Processed 2_p_5 with 7 items.
Processed 2_p_6 with 2 items.
Processed 2_p_7 with 21 items.
Processed 2_p_8 with 8 items.
Processed 2_p_9 with 23 items.
Processed 3_t_1 with 19 items.
Processed 3_t_2 with 11 items.
Processed 3_t_3 with 11 items.
Processed 3_t_4 with 15 items.
Processed 3_t_5 with 11 items.
Processed 4_t_1 with 32 items.
Processed 4_t_2 with 32 items.
Processed 4_t_3 with 56 items.
Processed 4_t_4 with 26 i

### Обучение сети на детекцию